In [1]:
# konlpy 설치
# !pip install konlpy

# flask_cors 설치
# !pip install flask_cors

# pymysql 설치
# !pip install pymysql

import pymysql

from flask import Flask, jsonify, request
from flask_cors import CORS

# Komoran (한국어 토큰화) import
from konlpy.tag import Komoran
komoran = Komoran()

# 코사인 유사도 import
from sklearn.metrics.pairwise import cosine_similarity

# ChatGPT API 사용 위한 라이브러리 import, api/client 정의
from openai import OpenAI
api = 'api_key'
client = OpenAI(api_key=api)

from sklearn.feature_extraction.text import TfidfVectorizer

# Papago API 사용 위한 라이브러리 import, id/secret 정의
import os
import sys
import urllib.request
client_id = "client_id"
client_secret = "client_secret"

import datetime

# 현재 날짜와 시간을 가져옴(DB insert 위함)
current_datetime = datetime.datetime.now()

In [2]:
# 영어 문장 정제 메서드 (오탈자 수정)

def en_correction(en_str):
    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": "The task is to correct the typographical errors in the sentences provided. Never change the meaning of a sentence."
        },
        {
            "role": "user",
            "content": en_str
        }
    ],
    temperature=0.5,    # 다양성의 정도(0~2) - 값이 크면 창의적인 대답, 값이 작으면 일관적인 대답
    max_tokens=500,
    top_p=0.1             # 문장 생성 시 다음 토큰 선택 과정에서 확률 분포 제어(0~1) - 0.1이면 상위 10%의 토큰 사용
    )
    return response.choices[0].message.content.strip()

In [3]:
# 한국어 문장 정제 메서드 (오탈자 수정)

def ko_correction(ko_str):
    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": "제공되는 문장의 오탈자만을 고치는 것이 작업입니다. 문장의 의미를 절대 바꾸지 마세요."
        },
        {
            "role": "user",
            "content": ko_str
        }
    ],
    temperature=0.1,    # 다양성의 정도(0~2) - 값이 크면 창의적인 대답, 값이 작으면 일관적인 대답
    max_tokens=500,
    top_p=0.1             # 문장 생성 시 다음 토큰 선택 과정에서 확률 분포 제어(0~1) - 0.1이면 상위 10%의 토큰 사용
    )
    return response.choices[0].message.content.strip()

In [4]:
# 친근한 문장으로 바꿔주는 메서드(영어)

def en_pretty(en_str):
    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": "The task is to turn the sentences provided into those that feel familiar. Never change the meaning of a sentence."
        },
        {
            "role": "user",
            "content": en_str
        }
    ],
    temperature=0.5,    # 다양성의 정도(0~2) - 값이 크면 창의적인 대답, 값이 작으면 일관적인 대답
    max_tokens=500,
    top_p=0.5             # 문장 생성 시 다음 토큰 선택 과정에서 확률 분포 제어(0~1) - 0.1이면 상위 10%의 토큰 사용
    )
    return response.choices[0].message.content.strip()

In [5]:
# 친근한 문장으로 바꿔주는 메서드(한국어)

def ko_pretty(ko_str):
    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": "제공되는 문장을 친근한 느낌의 문장으로 바꾸는 것이 작업입니다. 문장의 의미를 절대 바꾸지 마세요."
        },
        {
            "role": "user",
            "content": ko_str
        }
    ],
    temperature=0.5,    # 다양성의 정도(0~2) - 값이 크면 창의적인 대답, 값이 작으면 일관적인 대답
    max_tokens=500,
    top_p=0.5             # 문장 생성 시 다음 토큰 선택 과정에서 확률 분포 제어(0~1) - 0.1이면 상위 10%의 토큰 사용
    )
    return response.choices[0].message.content.strip()

In [6]:
# DB 연결
db = pymysql.connect(host='host', port='port', user='user', passwd='passwd', db='db', charset='utf8')
cursor = db.cursor()

In [7]:
# 한글 / 영어 구분 코드

def detect_language(text):
    # 한글 유니코드 범위
    korean_range = range(44032, 55204)

    # 영어 알파벳 범위
    english_range = list(range(65, 91)) # 대문자
    english_range += list(range(97, 123)) # 소문자

    is_korean = any(ord(char) in korean_range for char in text)
    is_english = any(ord(char) in english_range for char in text)

    if is_korean and not is_english:
        return "한글"
    elif is_english and not is_korean:
        return "영어"
    elif is_korean and is_english:
        return "혼합"
    else:
        return "기타"

In [8]:
stopwords = ["은", "는", "이", "가", "을", "를", "습니다", "에", "하", "고", "ㄴ가요", "나요", "ㄹ", "ㄴ", "ㅁ", ".", ",", "?", "대하", "아", "았", "었", "ㅂ니다", "어야",
            "드리", "과", "와", "의", "는데", "싶", "ㄴ데", "어요", "받", "어", "주", "시", "수", "은데", "면", "아야", "궁금", "알", "되", "을까요", "것", "데", "알리",
            "ㄹ까요", "지만", "아도", "걸", "라", "지", "는지", "은가요", "는가요", "에요", "는데요", "려면", "에서", "로", "요", "아요"]

In [9]:
# 한국어 질문 리스트 만들기(DB에 있는 한국어 질문들 가져오기)

cursor.execute("select * from QUESTION_KO")
q_ko_tuple = cursor.fetchall()

# 여기에 담기도록
Q_KO_list = []

for i in q_ko_tuple:
    Q_KO_list.append(i[4])

# 토큰화
contents_tokens = [komoran.morphs(row) for row in Q_KO_list]

# 불용어 제거
contents_no_stop = []

for i in range(len(contents_tokens)):
    result = [word for word in contents_tokens[i] if not word in stopwords]
    contents_no_stop.append(result)
    
# print(contents_no_stop)

# 토큰화 후 문장으로 만들기
K_contents_for_vectorize = []
for content in contents_no_stop:
    sentence = ''
    for word in content:
        sentence = sentence + " " + word
    K_contents_for_vectorize.append(sentence)

# 벡터화
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(K_contents_for_vectorize)
num_samples, num_features = X.shape
print(num_samples, num_features)

# num_samples: 처리한 문서(문장)의 수
# num_features: 각 단어의 등장 여부를 표현하는 특성의 수(사용된 고유한 단어의 수)

275 200


In [10]:
# 영어 질문 리스트 만들기(DB에 있는 영어 질문들 가져오기)

cursor.execute("select * from QUESTION_EN")
q_en_tuple = cursor.fetchall()

# 여기에 담기도록
Q_EN_list = []

for i in q_en_tuple:
    Q_EN_list.append(i[4])

In [11]:
# 한국어 → 영어 번역 메서드

def ko_to_en(str):
    encText = urllib.parse.quote(str)
    data = "source=ko&target=en&text=" + encText
    url = "https://naveropenapi.apigw.ntruss.com/nmt/v1/translation"
    request = urllib.request.Request(url)
    request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
    request.add_header("X-NCP-APIGW-API-KEY",client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        translated = response_body.decode('utf-8')
    else:
        print("Error Code:" + rescode)

    import json
    translated = json.loads(translated)

    translation = translated["message"]["result"]["translatedText"]
    return translation

In [12]:
# 영어 → 한국어 번역 메서드

def en_to_ko(str):
    encText = urllib.parse.quote(str)
    data = "source=en&target=ko&text=" + encText
    url = "https://naveropenapi.apigw.ntruss.com/nmt/v1/translation"
    request = urllib.request.Request(url)
    request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
    request.add_header("X-NCP-APIGW-API-KEY",client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        translated = response_body.decode('utf-8')
    else:
        print("Error Code:" + rescode)

    import json
    translated = json.loads(translated)

    translation = translated["message"]["result"]["translatedText"]
    return translation

In [13]:
########################## FLASK 연결 ##########################


# answer(리액트로 돌려줄 값) 초기화
answer = "answer"

app = Flask(__name__)
CORS(app)

@app.route('/send_data', methods=['POST'])
def send_data():
    
    # 사용자 입력 문장, 휴대폰 번호 받아옴
    new_post = request.json['chat']
    phone = request.json['phone']
    
    # 받은 휴대폰 번호가 공백이라면 "Anonymous"로 처리
    if(phone == ""):
        phone = "Anonymous"
    
    # 휴대폰 번호 확인
    print("phone: ", phone)

    # 받아온 휴대폰 번호를 통해 USER_IDX 가져오기
    cursor.execute("select USER_IDX from USER where USER_PHONE = %s", phone)
    user_idx = cursor.fetchone()
    
    
    # 입력 문장의 단어(공백 기준) 수 측정
    word_count = len(new_post.split())

    
    # 한글 / 영어 판단
    language = detect_language(new_post)
    print("입력 언어: ", language)

    ### 1. 사용자가 입력한 문장이 한글 문장이라면
    if(language == "한글"):
        print("입력 문장: ", new_post)
        new_post = ko_correction(new_post)
        print("정제된 문장: ", new_post)
        
        # 토큰화
        new_post_tokens = [komoran.morphs(new_post)]
        
        # 불용어 제거
        contents_no_stop = []

        for i in range(len(new_post_tokens)):
            result = [word for word in new_post_tokens[i] if not word in stopwords]
            contents_no_stop.append(result)
        
        # 토큰화 후 문장으로 만들기
        new_post_for_vectorize = []
        for content in contents_no_stop:
            sentence = ''
            for word in content:
                sentence = sentence + " " + word
            new_post_for_vectorize.append(sentence)
        
        # 벡터화
        new_post_vec = vectorizer.transform(new_post_for_vectorize)

        # 코사인 유사도 분석 (한국어)
        best_doc = None
        best_dist = -1
        best_i = None

        for i in range(0, num_samples):
            cos_similar = cosine_similarity(X[i:i+1], new_post_vec)
            if cos_similar > best_dist:
                best_dist = cos_similar
                best_i = i
                
        # 유사도가 너무 작을 때(best_dist < 0.5일 때) 처리하기
        if(best_dist < 0.5):
            print("유사도: ", best_dist)
            
            # 인덱스가 0번인 답변 출력
            cursor.execute("select * from ANSWER_KO where A_KO_IDX = 0")
            answer = cursor.fetchone()
            answer = answer[1]
            
            # 채팅 로그 테이블에 추가
            cursor.execute("insert into CHATTING_KO(C_KO_Q, C_KO_A_IDX, C_KO_USER_IDX, C_KO_TIME) values(%s, %s, %s, %s)", (new_post, 0, user_idx[0], current_datetime,))
            db.commit()

        else:
            print("============================================")
            print("== Best %i with dist = %.2f"%(best_i, best_dist))
            print('비교 문장: ', new_post)
            print('가장 유사한 문장: ', Q_KO_list[best_i])  # 가장 유사한 문장 출력 (테이블의 인덱스는 [best_i + 1])

            # inner join으로 답변(한국어) 찾아오기
            cursor.execute("select * from ANSWER_KO inner join QUESTION_KO on ANSWER_KO.A_KO_IDX = QUESTION_KO.Q_KO_A_IDX where QUESTION_KO.Q_KO_IDX = %s", (best_i + 1,))
            a_ko = cursor.fetchone()
            print("답변 : ", a_ko[1])
            
            if(a_ko[0] == 28):
                answer = a_ko[1]
            else:
                answer = ko_pretty(a_ko[1])
                print("친근한 답변: ", answer)

            cursor.execute("update ANSWER_KO set A_KO_COUNT = A_KO_COUNT + 1 where A_KO_IDX = %s", (a_ko[0],))
            db.commit()

            cursor.execute("insert into CHATTING_KO(C_KO_Q, C_KO_A_IDX, C_KO_USER_IDX, C_KO_TIME) values(%s, %s, %s, %s)", (new_post, a_ko[0], user_idx[0], current_datetime,))
            db.commit()

    ### 2. 사용자가 입력한 문장이 영어 문장이라면
    elif(language == "영어"):

        # 영어 질문 리스트(Q_EN_list)에 해당 문장이 있는지 확인
        same = 0
        same_idx = 0

        for i in range(0, len(Q_EN_list)):
            if(new_post == Q_EN_list[i]):
                same = 1
                same_idx = i
                break

        # 해당 문장이 있다면
        if (same == 1):
            print("들어온 문장과 똑같은 문장이 존재합니다.")
            # inner join으로 답변(영어) 찾아오기
            cursor.execute("select * from ANSWER_EN inner join QUESTION_EN on ANSWER_EN.A_EN_IDX = QUESTION_EN.Q_EN_A_IDX where QUESTION_EN.Q_EN_IDX = %s", (same_idx + 1,))
            a_en = cursor.fetchone()
            print("가져온 답변: ", a_en[1])
            cursor.execute("update ANSWER_EN set A_EN_COUNT = A_EN_COUNT + 1 where A_EN_IDX = %s", (a_en[0],))
            db.commit()
            
            # 친근한 답변으로 바꾸기
            answer = en_pretty(a_en[1])
            print("친근한 답변: ", answer)
            
            # 채팅 로그 테이블에 추가
            cursor.execute("insert into CHATTING_EN(C_EN_Q, C_EN_A_IDX, C_EN_USER_IDX, C_EN_TIME) values(%s, %s, %s, %s)", (new_post, a_en[0], user_idx[0], current_datetime))
            db.commit()

        # 해당 문장이 없다면
        else:
            print("처음 들어온 문장입니다.")

            # 영어 질문을 한국어로 번역 수행
            ko_translated_text = en_to_ko(new_post)
            print("들어온 질문 한국어로 번역: ", ko_translated_text)
            
            # 문장 정제 과정
            ko_translated_text = ko_correction(ko_translated_text)
            print("정제된 문장: ", ko_translated_text)
            
            # 번역/정제된 문장 토큰화
            new_post_tokens = [komoran.morphs(ko_translated_text)]
            
            # 불용어 제거
            contents_no_stop = []

            for i in range(len(new_post_tokens)):
                result = [word for word in new_post_tokens[i] if not word in stopwords]
                contents_no_stop.append(result)

            # 토큰화 후 문장으로 만들기
            new_post_for_vectorize = []
            for content in contents_no_stop:
                sentence = ''
                for word in content:
                    sentence = sentence + " " + word
                new_post_for_vectorize.append(sentence)
            
            # 벡터화
            new_post_vec = vectorizer.transform(new_post_for_vectorize)

            # 코사인 유사도 분석 (한국어)
            best_doc = None
            best_dist = -1
            best_i = None

            for i in range(0, num_samples):
                cos_similar = cosine_similarity(X[i:i+1], new_post_vec)
                if cos_similar > best_dist:
                    best_dist = cos_similar
                    best_i = i
                    
            # 유사도가 너무 작을 때(best_dist < 0.5일 때) 처리하기
            if(best_dist < 0.5):
                print("유사도: ", best_dist)
                
                # 인덱스가 0번인 답변 출력
                cursor.execute("select * from ANSWER_EN where A_EN_IDX = 0")
                answer = cursor.fetchone()
                
                print("유사도가 너무 작을 때 답변(리액트로 넘겨줄 값): ", answer[1])
                answer = answer[1]
                
                # 채팅 로그 테이블에 추가
                cursor.execute("insert into CHATTING_EN(C_EN_Q, C_EN_A_IDX, C_EN_USER_IDX, C_EN_TIME) values(%s, %s, %s, %s)", (new_post, 0, user_idx[0], current_datetime))
                db.commit()
            else:
                # 가장 유사한 문장 출력 (한국어 질문 테이블의 인덱스는 [best_i + 1])
                print("============================================")
                print("== Best %i with dist = %.2f"%(best_i, best_dist))
                print('-->', new_post)
                print('---->', Q_KO_list[best_i])

                # 가장 유사한 한국어 질문 행 가져오기
                # 컬럼 순서 : Q_KO_IDX, Q_KO_DEP1, Q_KO_DEP2, Q_KO_DEP3, Q_KO_QUESTION, Q_KO_A_IDX
                cursor.execute("select * from QUESTION_KO where Q_KO_IDX = %s", (best_i + 1,))
                q_ko = cursor.fetchone()
                print("가장 유사한 한국어 질문: ", q_ko)
                
                # 위의 질문(한국어)에 연결된 답변(한국어) 가져오기
                cursor.execute("select * from ANSWER_KO where A_KO_IDX = %s", (q_ko[5],))
                a_ko = cursor.fetchone()
                print("연결된 한국어 답변: ", a_ko)
                
                # 예외 처리
                # a_ko[0] : 한국어 답변 인덱스
                if(a_ko[0] == 28):
                    answer = "Hello, this is TokTalk! How can I help you?"
                    print("인삿말 예외 처리")
                    
                    # 채팅 로그 테이블에 추가
                    cursor.execute("insert into CHATTING_EN(C_EN_Q, C_EN_A_IDX, C_EN_USER_IDX, C_EN_TIME) values(%s, %s, %s, %s)", (new_post, 28, user_idx[0], current_datetime))
                    db.commit()
                elif(a_ko[0] == 24):
                    answer = "■ If an error occurs while using the system, you can attach the company name, error content, contact information, and a captured copy of the error screen to cert.k-startup@kised.or.kr , or you can solve the problem through the customer service center. Contact customer service: https://cert.k-startup.go.kr/usr/bbs/selectContent.do?tempValue=0504"
                    print("24번 답변(시스템 에러) 예외 처리")
                    
                    # 채팅 로그 테이블에 추가
                    cursor.execute("insert into CHATTING_EN(C_EN_Q, C_EN_A_IDX, C_EN_USER_IDX, C_EN_TIME) values(%s, %s, %s, %s)", (new_post, 24, user_idx[0], current_datetime))
                    db.commit()
                elif(a_ko[0] == 6):
                    answer = "■ How to issue confirmation for start-up companies How to issue confirmation for start-up companies is as follows. 1. Start-up company verification system Membership registration 2. At the top of the homepage [My Page] - [My Company Information] - [Company Registration] 3. At the top of the homepage [Start-up company self-diagnosis] - [Self-diagnosis] 4. At the top of the homepage [Issuance of confirmation for start-up companies] - [Application for issuance of confirmation] to apply ( ※ You can check the required documents on the application form screen.) 5. Examination of whether to start-up companies ( ※ The examination will be conducted within 10 days from the date of receipt after the application is submitted, so please refer to it.) 6. Issuance ( ※ After the examination is completed, we will check whether to issue a confirmation of SMEs through data connection, so please refer to it.) Start-up company verification system: http://cert.k-startup.go.kr"
                    print("6번 답변(창업기업 확인서 발급방법) 예외 처리")
                elif(word_count == 1):
                    answer = "The sentence is too short. Please tell me more accurately."
                    print("질문이 한 단어일 때 처리")
                else:
                    # 가져온 답변(한국어) 영어로 번역하기
                    en_translated_text = ko_to_en(a_ko[1])
                    print("연결된 한국어 답변 영어로 번역: ", en_translated_text)

                    en_translated_text = en_correction(en_translated_text)
                    print("정제된 영어 문장: ", en_translated_text)

                    answer = en_pretty(en_translated_text)
                    print("친근한 답변(리액트로 넘겨줄 값): ", answer)

                    # 인덱스가 a_ko[0]인 영어 답변 있는지 확인 (이미 ANSWER_EN 테이블에 insert된 답변인지 확인)
                    cursor.execute("select * from ANSWER_EN where A_EN_IDX = %s", (a_ko[0],))
                    check = cursor.fetchone()
                    print(check)

                    if check is None:
                        # ANSWER_EN 테이블에 없다면? -> 번역된 영어 답변 insert
                        print("처음 출력하는 인덱스")
                        cursor.execute("insert into ANSWER_EN(A_EN_IDX, A_EN_ANSWER, A_EN_COUNT) values(%s, %s, %s)", (a_ko[0], en_translated_text, 1,))
                        db.commit()
                        
                        # 채팅 로그 테이블에 추가
                        cursor.execute("insert into CHATTING_EN(C_EN_Q, C_EN_A_IDX, C_EN_USER_IDX, C_EN_TIME) values(%s, %s, %s, %s)", (new_post, a_ko[0], user_idx[0], current_datetime))
                        db.commit()
                    else:
                        # ANSWER_EN 테이블에 있다면? -> COUNT만 업데이트
                        print("이미 출력된 인덱스")
                        cursor.execute("update ANSWER_EN set A_EN_COUNT = A_EN_COUNT + 1 where A_EN_IDX = %s", (a_ko[0],))
                        db.commit()
                        
                        # 채팅 로그 테이블에 추가
                        cursor.execute("insert into CHATTING_EN(C_EN_Q, C_EN_A_IDX, C_EN_USER_IDX, C_EN_TIME) values(%s, %s, %s, %s)", (new_post, a_ko[0], user_idx[0], current_datetime))
                        db.commit()

                # 영어 질문 insert 위해서 depth1, 2, 3 번역하기(한국어 -> 영어)
                dep1 = ko_to_en(q_ko[1])
                dep2 = ko_to_en(q_ko[2])
                dep3 = ko_to_en(q_ko[3])

                print("DEPTH: ", dep1, dep2, dep3)

                # 영어 질문 insert
                cursor.execute("insert into QUESTION_EN(Q_EN_DEP1, Q_EN_DEP2, Q_EN_DEP3, Q_EN_QUESTION, Q_EN_A_IDX) values(%s, %s, %s, %s, %s)", (dep1, dep2, dep3, new_post, a_ko[0],))
                db.commit()
                print("영어 질문 insert 완료")


    # 혼합 / 기타일 경우
    else:
        # 인덱스가 0번인 답변 출력
        cursor.execute("select * from ANSWER_EN where A_EN_IDX = 0")
        answer = cursor.fetchone()
        answer = answer[1]
    
    
    
    print("최종 답변: ", answer)
    print("끝***********************************************************************끝")
    
    # 리액트로 답변 넘겨주기
    data = {'message': answer}
    return jsonify(data)


if __name__ == '__main__':
    app.run(host='127.0.0.1', port=8089)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8089
Press CTRL+C to quit
127.0.0.1 - - [13/Apr/2024 14:21:36] "OPTIONS /send_data HTTP/1.1" 200 -


phone:  Anonymous
입력 언어:  한글
입력 문장:  안녕


127.0.0.1 - - [13/Apr/2024 14:21:37] "POST /send_data HTTP/1.1" 200 -


정제된 문장:  안녕하세요.
== Best 274 with dist = 1.00
비교 문장:  안녕하세요.
가장 유사한 문장:  안녕하세요
답변 :  안녕하세요, 톡톡입니다! 무엇을 도와드릴까요?
최종 답변:  안녕하세요, 톡톡입니다! 무엇을 도와드릴까요?
끝***********************************************************************끝


...

정제된 문장:  안녕하세요.
== Best 274 with dist = 1.00
비교 문장:  안녕하세요.
가장 유사한 문장:  안녕하세요
답변 :  안녕하세요, 톡톡입니다! 무엇을 도와드릴까요?
최종 답변:  안녕하세요, 톡톡입니다! 무엇을 도와드릴까요?
끝***********************************************************************끝


127.0.0.1 - - [13/Apr/2024 14:26:15] "OPTIONS /send_data HTTP/1.1" 200 -


phone:  Anonymous
입력 언어:  한글
입력 문장:  안녕


127.0.0.1 - - [13/Apr/2024 14:26:16] "POST /send_data HTTP/1.1" 200 -


정제된 문장:  안녕하세요.
== Best 274 with dist = 1.00
비교 문장:  안녕하세요.
가장 유사한 문장:  안녕하세요
답변 :  안녕하세요, 톡톡입니다! 무엇을 도와드릴까요?
최종 답변:  안녕하세요, 톡톡입니다! 무엇을 도와드릴까요?
끝***********************************************************************끝


In [40]:
db.commit()

In [69]:
db.close()